In [359]:
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,roc_auc_score,roc_curve

In [445]:
data = pd.read_csv('sales-data.csv',parse_dates=[0])
data.drop(['Mobile',"EMAIL"],axis=1,inplace=True)
data.dropna(inplace=True)
data.reset_index(inplace=True,drop=True)
data.head(2)

,Created,Product_ID,Source,Sales_Agent,Location,Delivery_Mode,Status
0,2018-11-13 18:51:00,9.0,Live Chat-Google Organic,Sales-Agent-3,Bangalore,Mode-1,Potential
1,2018-11-13 18:48:00,19.0,Call,Sales-Agent-4,Other Locations,Mode-5,Potential


In [ ]:
#for i in range(data.shape[0]):
#    data.loc[i,'day']=data.loc[i,'Created'].isoweekday()
#data.day = data.day.astype('int')

In [446]:
Counter(data.Status)

Counter({'CONVERTED': 834,
         'In Progress Negative': 625,
         'In Progress Positive': 642,
         'Junk Lead': 1532,
         'Just Enquiry': 751,
         'LOST': 439,
         'Long Term': 644,
         'Not Responding': 1119,
         'Open': 19,
         'Potential': 705,
         'converted': 18})

In [450]:
Counter(data.Product_ID).most_common

<bound method Counter.most_common of Counter({18.0: 1706, 15.0: 1501, 19.0: 1188, 9.0: 990, 27.0: 737, 5.0: 484, 10.0: 168, 1.0: 104, 20.0: 101, 25.0: 90, 21.0: 65, 2.0: 38, 12.0: 36, 26.0: 31, 14.0: 27, 11.0: 12, 22.0: 8, 3.0: 7, 6.0: 7, 8.0: 6, 17.0: 6, 13.0: 4, 16.0: 3, 24.0: 2, 23.0: 2, 0.0: 2, 28.0: 1, 7.0: 1, 4.0: 1})>

In [449]:
#Counter(data.Location)

In [451]:
#data = data[data.Location=='Bangalore']
data = data[(data.Product_ID==18.0)|(data.Product_ID==15.0)|(data.Product_ID==19.0)|(data.Product_ID==9.0)|(data.Product_ID==27.0)]

In [452]:
data.loc[(data.Status=='CONVERTED'),'Status']='GOOD'
data.loc[(data.Status=='In Progress Positive'),'Status']='GOOD'
data.loc[(data.Status=='converted'),'Status']='GOOD'
data.loc[(data.Status=='Potential'),'Status']='GOOD'
data.loc[(data.Status=='Long Term'),'Status']='GOOD'


data.loc[(data.Status=='Open'),'Status']='BAD'
data.loc[(data.Status=='In Progress Negative'),'Status']='BAD'
data.loc[(data.Status=='Junk Lead'),'Status']='BAD'
data.loc[(data.Status=='Just Enquiry'),'Status']='BAD'
data.loc[(data.Status=='LOST'),'Status']='BAD'
data.loc[(data.Status=='Not Responding'),'Status']='BAD'

In [453]:
data.head()

,Created,Product_ID,Source,Sales_Agent,Location,Delivery_Mode,Status
0,2018-11-13 18:51:00,9.0,Live Chat-Google Organic,Sales-Agent-3,Bangalore,Mode-1,GOOD
1,2018-11-13 18:48:00,19.0,Call,Sales-Agent-4,Other Locations,Mode-5,GOOD
2,2018-11-13 17:09:00,18.0,Website,Sales-Agent-11,Trivandrum,Mode-1,GOOD
3,2018-11-13 16:49:00,15.0,Website,Sales-Agent-7,Hyderabad,Mode-1,BAD
4,2018-11-13 16:37:00,18.0,Call,Sales-Agent-7,Bangalore,Mode-1,GOOD


In [454]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
data.Product_ID = enc.fit_transform(data.Product_ID)
data.Source = enc.fit_transform(data.Source)
data.Sales_Agent = enc.fit_transform(data.Sales_Agent)
data.Location = enc.fit_transform(data.Location)
data.Delivery_Mode = enc.fit_transform(data.Delivery_Mode)
#data.day = enc.fit_transform(X.day)
data.Status = enc.fit_transform(data.Status)
#X = pd.get_dummies(X)
data.head(1)

,Created,Product_ID,Source,Sales_Agent,Location,Delivery_Mode,Status
0,2018-11-13 18:51:00,0,16,4,1,0,1


In [455]:
data.corr()

,Product_ID,Source,Sales_Agent,Location,Delivery_Mode,Status
Product_ID,1.000000,0.151328,0.006681,-0.166249,-0.149998,0.154903
Source,0.151328,1.000000,-0.023877,0.028338,-0.177660,-0.022442
Sales_Agent,0.006681,-0.023877,1.000000,-0.153392,-0.237714,0.132763
Location,-0.166249,0.028338,-0.153392,1.000000,0.423779,-0.343816
Delivery_Mode,-0.149998,-0.177660,-0.237714,0.423779,1.000000,-0.257424
Status,0.154903,-0.022442,0.132763,-0.343816,-0.257424,1.000000


In [456]:
X = data.iloc[:,[1,2,3,4,5]]
y = data.Status
X.head()

,Product_ID,Source,Sales_Agent,Location,Delivery_Mode
0,0,16,4,1,0
1,3,2,5,10,4
2,2,23,1,13,0
3,1,23,8,6,0
4,2,2,8,1,0


In [457]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1)

In [458]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=80,max_depth=15,gamma=5,learning_rate=0.01)
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.714207947741
             precision    recall  f1-score   support

          0       0.74      0.83      0.78      1135
          1       0.66      0.52      0.58       702

avg / total       0.71      0.71      0.71      1837



col_0,0,1
Status,,
0,944,191
1,334,368


In [439]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_sample(X_train,y_train)

In [443]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=250,max_depth=2,gamma=3,learning_rate=0.001)
model.fit(X_train_smote,y_train_smote)
y_predict = model.predict(X_test.values)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.624387588459
             precision    recall  f1-score   support

          0       0.89      0.44      0.59      1135
          1       0.50      0.91      0.65       702

avg / total       0.75      0.62      0.62      1837



col_0,0,1
Status,,
0,505,630
1,60,642


In [401]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

parameters = {'kernel': ['rbf'], 'gamma': [0.1,1,5],
                     'C': [ 0.1,1,100, 1000,]}
                    
grid_model = RandomizedSearchCV(SVC(), parameters)
grid_model.fit(X_train_smote, y_train_smote)
print(grid_model.best_score_)
print(grid_model.best_params_)
y_predict = grid_model.predict(X_test.values)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.718326230143
{'kernel': 'rbf', 'gamma': 0.1, 'C': 100}
0.697876973326
             precision    recall  f1-score   support

          0       0.78      0.70      0.74      1132
          1       0.59      0.69      0.64       705

avg / total       0.71      0.70      0.70      1837



col_0,0,1
Status,,
0,796,336
1,219,486


In [404]:
from sklearn.svm import SVC
model = SVC(kernel='rbf',gamma=5,C=10000)
model.fit(X_train_smote, y_train_smote)
y_predict = model.predict(X_test)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))

pd.crosstab(y_test,y_predict)

0.646706586826
             precision    recall  f1-score   support

          0       0.81      0.56      0.66      1132
          1       0.53      0.78      0.63       705

avg / total       0.70      0.65      0.65      1837



col_0,0,1
Status,,
0,637,495
1,154,551


In [405]:
parameters = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

parameters = {
 'max_depth': [200,300,None],
 'min_samples_leaf': [2,5,10],
 'n_estimators': [200,500,1000]}


In [406]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

grid_model = RandomizedSearchCV(RandomForestClassifier(), parameters)
grid_model.fit(X_train_smote, y_train_smote)
print(grid_model.best_score_)
print(grid_model.best_params_)
y_predict = grid_model.predict(X_test.values)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.726462611391
{'n_estimators': 200, 'min_samples_leaf': 2, 'max_depth': 200}
0.715841045182


col_0,0,1
Status,,
0,846,286
1,236,469


In [407]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

parameters = {
 'n_estimators': [80,100],
 'max_depth': [2,3,4],
 'gamma': [0.1,1],
 'learning_rate':[0.01,0.1,0.5],
 'colsample_bytree':[1],
 'subsample' : [1]}

grid_model = GridSearchCV(XGBClassifier(), parameters)
eval_set = [(X_train_smote, y_train_smote), (X_test.values, y_test.values)]
eval_metric = ["auc","error"]
%time grid_model.fit(X_train_smote,y_train_smote,eval_metric=eval_metric, eval_set=eval_set, verbose=True)

print(grid_model.best_score_)
print(grid_model.best_params_)
y_predict = grid_model.predict(X_test.values)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)



[0]	validation_0-auc:0.734393	validation_0-error:0.329136	validation_1-auc:0.754721	validation_1-error:0.376157
[1]	validation_0-auc:0.734393	validation_0-error:0.329136	validation_1-auc:0.754721	validation_1-error:0.376157
[2]	validation_0-auc:0.734393	validation_0-error:0.329136	validation_1-auc:0.754721	validation_1-error:0.376157
[3]	validation_0-auc:0.734393	validation_0-error:0.329136	validation_1-auc:0.754721	validation_1-error:0.376157
[4]	validation_0-auc:0.734393	validation_0-error:0.329136	validation_1-auc:0.754721	validation_1-error:0.376157
[5]	validation_0-auc:0.734393	validation_0-error:0.329136	validation_1-auc:0.754721	validation_1-error:0.376157
[6]	validation_0-auc:0.734393	validation_0-error:0.329136	validation_1-auc:0.754721	validation_1-error:0.376157
[7]	validation_0-auc:0.734393	validation_0-error:0.329136	validation_1-auc:0.754721	validation_1-error:0.376157
[8]	validation_0-auc:0.734393	validation_0-error:0.329136	validation_1-auc:0.754721	validation_1-error:0

[73]	validation_0-auc:0.760059	validation_0-error:0.30957	validation_1-auc:0.777327	validation_1-error:0.334785
[74]	validation_0-auc:0.760059	validation_0-error:0.30957	validation_1-auc:0.777327	validation_1-error:0.334785
[75]	validation_0-auc:0.759513	validation_0-error:0.30957	validation_1-auc:0.777615	validation_1-error:0.334785
[76]	validation_0-auc:0.76007	validation_0-error:0.30957	validation_1-auc:0.777753	validation_1-error:0.334785
[77]	validation_0-auc:0.760604	validation_0-error:0.30957	validation_1-auc:0.777386	validation_1-error:0.334785
[78]	validation_0-auc:0.760614	validation_0-error:0.30957	validation_1-auc:0.777327	validation_1-error:0.334785
[79]	validation_0-auc:0.760986	validation_0-error:0.30957	validation_1-auc:0.777644	validation_1-error:0.334785
[0]	validation_0-auc:0.734393	validation_0-error:0.330492	validation_1-auc:0.754721	validation_1-error:0.292869
[1]	validation_0-auc:0.734393	validation_0-error:0.330492	validation_1-auc:0.754721	validation_1-error:0.

[66]	validation_0-auc:0.75829	validation_0-error:0.321387	validation_1-auc:0.772007	validation_1-error:0.310289
[67]	validation_0-auc:0.75829	validation_0-error:0.321387	validation_1-auc:0.772007	validation_1-error:0.310289
[68]	validation_0-auc:0.75829	validation_0-error:0.321387	validation_1-auc:0.772007	validation_1-error:0.310289
[69]	validation_0-auc:0.75829	validation_0-error:0.321387	validation_1-auc:0.772007	validation_1-error:0.310289
[70]	validation_0-auc:0.75829	validation_0-error:0.321387	validation_1-auc:0.772007	validation_1-error:0.310289
[71]	validation_0-auc:0.75829	validation_0-error:0.321387	validation_1-auc:0.772007	validation_1-error:0.310289
[72]	validation_0-auc:0.75829	validation_0-error:0.321387	validation_1-auc:0.772007	validation_1-error:0.310289
[73]	validation_0-auc:0.75829	validation_0-error:0.321387	validation_1-auc:0.772007	validation_1-error:0.310289
[74]	validation_0-auc:0.75829	validation_0-error:0.321387	validation_1-auc:0.772007	validation_1-error:0

[59]	validation_0-auc:0.752106	validation_0-error:0.331654	validation_1-auc:0.773657	validation_1-error:0.319543
[60]	validation_0-auc:0.7521	validation_0-error:0.331654	validation_1-auc:0.773658	validation_1-error:0.319543
[61]	validation_0-auc:0.752112	validation_0-error:0.331654	validation_1-auc:0.773639	validation_1-error:0.319543
[62]	validation_0-auc:0.752165	validation_0-error:0.331654	validation_1-auc:0.773623	validation_1-error:0.319543
[63]	validation_0-auc:0.752165	validation_0-error:0.331654	validation_1-auc:0.773623	validation_1-error:0.319543
[64]	validation_0-auc:0.752571	validation_0-error:0.331654	validation_1-auc:0.773934	validation_1-error:0.318998
[65]	validation_0-auc:0.752634	validation_0-error:0.331654	validation_1-auc:0.774236	validation_1-error:0.319543
[66]	validation_0-auc:0.752587	validation_0-error:0.331654	validation_1-auc:0.773957	validation_1-error:0.319543
[67]	validation_0-auc:0.752657	validation_0-error:0.331654	validation_1-auc:0.774239	validation_1-

[52]	validation_0-auc:0.758163	validation_0-error:0.309957	validation_1-auc:0.778737	validation_1-error:0.335329
[53]	validation_0-auc:0.758163	validation_0-error:0.309957	validation_1-auc:0.778737	validation_1-error:0.335329
[54]	validation_0-auc:0.758619	validation_0-error:0.309957	validation_1-auc:0.777376	validation_1-error:0.335329
[55]	validation_0-auc:0.758619	validation_0-error:0.309957	validation_1-auc:0.777376	validation_1-error:0.335329
[56]	validation_0-auc:0.758701	validation_0-error:0.309957	validation_1-auc:0.777463	validation_1-error:0.335329
[57]	validation_0-auc:0.758701	validation_0-error:0.30957	validation_1-auc:0.777463	validation_1-error:0.334785
[58]	validation_0-auc:0.758701	validation_0-error:0.309957	validation_1-auc:0.777463	validation_1-error:0.335329
[59]	validation_0-auc:0.7593	validation_0-error:0.30957	validation_1-auc:0.777539	validation_1-error:0.334785
[60]	validation_0-auc:0.758945	validation_0-error:0.30957	validation_1-auc:0.777332	validation_1-err

...

[60]	validation_0-auc:0.871545	validation_0-error:0.216389	validation_1-auc:0.794499	validation_1-error:0.288514
[61]	validation_0-auc:0.871545	validation_0-error:0.216389	validation_1-auc:0.794499	validation_1-error:0.288514
[62]	validation_0-auc:0.871545	validation_0-error:0.216389	validation_1-auc:0.794499	validation_1-error:0.288514
[63]	validation_0-auc:0.871545	validation_0-error:0.216389	validation_1-auc:0.794499	validation_1-error:0.288514
[64]	validation_0-auc:0.871545	validation_0-error:0.216389	validation_1-auc:0.794499	validation_1-error:0.288514
[65]	validation_0-auc:0.871545	validation_0-error:0.216389	validation_1-auc:0.794499	validation_1-error:0.288514
[66]	validation_0-auc:0.871545	validation_0-error:0.216389	validation_1-auc:0.794499	validation_1-error:0.288514
[67]	validation_0-auc:0.871545	validation_0-error:0.216389	validation_1-auc:0.794499	validation_1-error:0.288514
[68]	validation_0-auc:0.871545	validation_0-error:0.216389	validation_1-auc:0.794499	validation_

[33]	validation_0-auc:0.818617	validation_0-error:0.263851	validation_1-auc:0.796361	validation_1-error:0.310289
[34]	validation_0-auc:0.818617	validation_0-error:0.263851	validation_1-auc:0.796361	validation_1-error:0.310289
[35]	validation_0-auc:0.818617	validation_0-error:0.263851	validation_1-auc:0.796361	validation_1-error:0.310289
[36]	validation_0-auc:0.818617	validation_0-error:0.263851	validation_1-auc:0.796361	validation_1-error:0.310289
[37]	validation_0-auc:0.818617	validation_0-error:0.263851	validation_1-auc:0.796361	validation_1-error:0.310289
[38]	validation_0-auc:0.818617	validation_0-error:0.263851	validation_1-auc:0.796361	validation_1-error:0.310289
[39]	validation_0-auc:0.818617	validation_0-error:0.263851	validation_1-auc:0.796361	validation_1-error:0.310289
[40]	validation_0-auc:0.818617	validation_0-error:0.263851	validation_1-auc:0.796361	validation_1-error:0.310289
[41]	validation_0-auc:0.818617	validation_0-error:0.263851	validation_1-auc:0.796361	validation_

[6]	validation_0-auc:0.822119	validation_0-error:0.268501	validation_1-auc:0.800415	validation_1-error:0.302123
[7]	validation_0-auc:0.829135	validation_0-error:0.259008	validation_1-auc:0.803524	validation_1-error:0.293413
[8]	validation_0-auc:0.83056	validation_0-error:0.256102	validation_1-auc:0.804503	validation_1-error:0.286881
[9]	validation_0-auc:0.834335	validation_0-error:0.256296	validation_1-auc:0.805215	validation_1-error:0.291236
[10]	validation_0-auc:0.836465	validation_0-error:0.256683	validation_1-auc:0.803183	validation_1-error:0.290147
[11]	validation_0-auc:0.838927	validation_0-error:0.251259	validation_1-auc:0.804027	validation_1-error:0.285248
[12]	validation_0-auc:0.843275	validation_0-error:0.248353	validation_1-auc:0.803426	validation_1-error:0.285792
[13]	validation_0-auc:0.846845	validation_0-error:0.24351	validation_1-auc:0.803188	validation_1-error:0.290147
[14]	validation_0-auc:0.84977	validation_0-error:0.236343	validation_1-auc:0.802651	validation_1-error

[79]	validation_0-auc:0.876945	validation_0-error:0.21174	validation_1-auc:0.803048	validation_1-error:0.288514
CPU times: user 43.1 s, sys: 18.4 s, total: 1min 1s
Wall time: 46.9 s
0.726075164665
{'colsample_bytree': 1, 'gamma': 0.1, 'learning_rate': 0.5, 'max_depth': 3, 'n_estimators': 80, 'subsample': 1}
0.711486118672
             precision    recall  f1-score   support

          0       0.77      0.75      0.76      1132
          1       0.62      0.65      0.63       705

avg / total       0.71      0.71      0.71      1837



col_0,0,1
Status,,
0,851,281
1,249,456


In [441]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=300)
model.fit(X_train_smote,y_train_smote)
y_predict = model.predict(X_test.values)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.68426782798
             precision    recall  f1-score   support

          0       0.79      0.67      0.72      1135
          1       0.57      0.71      0.63       702

avg / total       0.71      0.68      0.69      1837



col_0,0,1
Status,,
0,758,377
1,203,499
